In [1]:
import pickle 
import pandas as pd 
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm
import torch.optim as optim

import wandb

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))

55685


In [4]:
#input layer list [81]*n

inputlayer = open( r"DATA/input_list.npy", 'rb')
inputlayerdf = pickle.load(inputlayer)
inputlayerdf

[[525.0,
  1,
  0.059672102206736355,
  0.0013038454946651171,
  0.06148571428571429,
  0.05890175351276232,
  0.0009527063639297911,
  0.059994277847454834,
  8.902000000000001,
  5.868779259777964,
  5,
  8,
  0.05848506493506494,
  0.0008907936012381001,
  0.06027272727272727,
  0.058515294245794845,
  0.0009500584460310687,
  0.06052134268110914,
  5.381818181818182,
  5.208435116581214,
  11,
  6,
  0.061256388082696595,
  0.0011419304507069529,
  0.06268354430379747,
  0.06081872302453238,
  0.0011619344086033771,
  0.062032575589083185,
  14.535555555555554,
  6.325172154003224,
  9,
  5,
  0.05816878881987578,
  0.0013401291879889708,
  0.059391304347826086,
  0.05788880463785036,
  0.00204291954498698,
  0.06056436558833152,
  5.805000000000001,
  4.601206363552932,
  4,
  2,
  0.05945284327323162,
  0.0007404203490765129,
  0.060310679611650486,
  0.05963151981179333,
  0.0007885805708782171,
  0.0606851668606789,
  9.5975,
  5.4755905313186695,
  4,
  7,
  0.0579619713904979

In [9]:
inputlayer = open( r"DATA/total_list.npy", 'rb')
inputlayerdf = pickle.load(inputlayer)
race_ids,inputs,classes = zip(*inputlayerdf)

In [10]:
#Creates inputs array and replaces NAN with -1

inputs_df = pd.DataFrame(inputs)
inputs_df

#removing nan from inputs and convert to float
inputs_df = pd.DataFrame(inputs)
inputs_df.fillna(value=-1,inplace = True)
inputs = inputs_df.values.tolist()
inputs = [[float(i) for i in j] for j in inputs]
inputs_df

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
0,525.0,1,0.059672,0.001304,0.061486,0.058902,0.000953,0.059994,8.902000,5.868779,...,3.0,0.059185,0.000966,0.060114,0.060028,0.001071,0.061021,8.994000,5.421091,5.0
1,525.0,1,0.057393,0.000713,0.057897,0.058254,0.001901,0.059598,5.725000,0.346482,...,6.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.0
2,425.0,5,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,7.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.0
3,450.0,4,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,2.0,0.058094,-1.000000,0.058094,0.059359,-1.000000,0.059359,7.160000,-1.000000,1.0
4,525.0,2,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,6.0,0.057400,-1.000000,0.057400,0.057336,-1.000000,0.057336,7.430000,-1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55680,311.0,6,0.058240,0.000886,0.062830,0.057738,0.001296,0.065532,5.833421,4.071322,...,8.0,0.057429,0.000817,0.059942,0.057428,0.000988,0.059546,5.491579,3.824814,19.0
55681,311.0,3,0.058245,0.001394,0.060900,0.057955,0.001668,0.060763,3.952791,3.464289,...,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0
55682,311.0,4,0.059310,0.000987,0.061300,0.058430,0.002275,0.061175,6.446000,5.471536,...,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0
55683,311.0,2,0.061174,0.000814,0.061961,0.061160,0.001134,0.062706,12.125000,5.341663,...,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0


In [11]:
REBUILD_DATA = True

class GRV():
    #class to store training data
    
    #reading data from pickle
    file = open( r"DATA/total_list.npy", 'rb')
    data = pickle.load(file)
    #seperate out classes from inputs
    raceIDs,inputs,classes = zip(*data)
    
    #removing nan from inputs and convert to float
    inputs_df = pd.DataFrame(inputs)
    inputs_df.fillna(value=-1,inplace = True)
    inputs = inputs_df.values.tolist()
    inputs = [[float(i) for i in j] for j in inputs]
    
    
    
    #data
    training_data = []

    def make_training_data(self):
        excluded = 0
        for i in range(len(inputs)):
            if len(classes[i]) == 8:
                self.training_data.append([np.array(inputs[i]),np.array(classes[i])])
            else:
                adjustedList = classes[i]+([8]*(8-len(classes[i])))
                self.training_data.append([np.array(inputs[i]),np.array(adjustedList)])
                if len(adjustedList) != 8:
                    print(adjustedList)
        np.save('training_data.npy', self.training_data)
        print("excluded = ", excluded)

if REBUILD_DATA:
    grv = GRV()
    grv.make_training_data()

C:\Users\Nick\.conda\envs\pytorch\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


excluded =  0


In [9]:
grv.training_data

[[array([ 5.25000000e+02,  1.00000000e+00,  5.96721022e-02,  1.30384549e-03,
          6.14857143e-02,  5.89017535e-02,  9.52706364e-04,  5.99942778e-02,
          8.90200000e+00,  5.86877926e+00,  5.00000000e+00,  8.00000000e+00,
          5.84850649e-02,  8.90793601e-04,  6.02727273e-02,  5.85152942e-02,
          9.50058446e-04,  6.05213427e-02,  5.38181818e+00,  5.20843512e+00,
          1.10000000e+01,  6.00000000e+00,  6.12563881e-02,  1.14193045e-03,
          6.26835443e-02,  6.08187230e-02,  1.16193441e-03,  6.20325756e-02,
          1.45355556e+01,  6.32517215e+00,  9.00000000e+00,  5.00000000e+00,
          5.81687888e-02,  1.34012919e-03,  5.93913043e-02,  5.78888046e-02,
          2.04291954e-03,  6.05643656e-02,  5.80500000e+00,  4.60120636e+00,
          4.00000000e+00,  2.00000000e+00,  5.94528433e-02,  7.40420349e-04,
          6.03106796e-02,  5.96315198e-02,  7.88580571e-04,  6.06851669e-02,
          9.59750000e+00,  5.47559053e+00,  4.00000000e+00,  7.00000000e+00,

In [12]:
# Defines the nueral network:
# 81 -> 64 -> 32 -> 8

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(81, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 8)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x #nn.Softmax(x, dim=1)
        
        
net = Net().to(device)
print(net)    
#define tensors

training_data = grv.training_data

X = torch.Tensor([i[0] for i in training_data])
Y = torch.Tensor([i[1] for i in training_data])

#Generate winner only class
Y_w = []
for i in Y:
    n = np.zeros(8)
    index = torch.argmin(i)
    n[index] = float(1)
    Y_w.append(n)
    
Y_w = torch.tensor([i for i in Y_w])
    
VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)
print(val_size)
train_X = X[:-val_size]
train_y = Y_w[:-val_size]

test_X = X[-val_size:]
test_y = Y_w[-val_size:]
print(len(train_X), len(test_X))

test_X = test_X.to(device)
test_y = test_y.to(device)
    
def train(net):
    BATCH_SIZE = 100
    EPOCHS = 1000
    loss_function = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.05)

    for epoch in tqdm.tqdm(range(EPOCHS)):
        for i in range(0, len(train_X), BATCH_SIZE): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_X[i:i+BATCH_SIZE]
            batch_y = train_y[i:i+BATCH_SIZE]

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            net.zero_grad()
            
            m = nn.LogSoftmax(dim = 1)
            outputs = m(net(batch_X))
            loss = loss_function(outputs, batch_y.float())
            loss.backward()
            optimizer.step()    # Does the update



        #print(f"Epoch: {epoch}. Loss: {loss}")
        #test(net)
        if (epoch%100 == 0):
            print(f"Epoch: {epoch}. Loss: {loss}")
            test(net)
            
def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm.tqdm(range(len(test_X))):
            #print("test_y[i] \n", test_y[i], "\noutput y =", net(test_X[i]))
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_X[i]).to(device)  # returns a list, 
            #print(net_out)
            predicted_class = torch.argmax(net_out)
            #print("predicted_class = ", predicted_class,"\nreal class = ", real_class)
            
            #print(predicted_class)
            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total, 3))

Net(
  (fc1): Linear(in_features=81, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=8, bias=True)
)
5568
50117 5568


In [13]:
#Generate winner only class
Y_w = []
for i in Y:
    n = np.zeros(8)
    index = torch.argmin(i)
    n[index] = float(1)
    Y_w.append(n)
Y_w = torch.tensor([i for i in Y_w])
Y_w[1:10]

tensor([[0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.]], dtype=torch.float64)

In [7]:
#indexed win only
Y_wi = []
for i in Y:
    #print(torch.argmin(i))
    Y_wi.append(torch.argmin(i))
Y_w = torch.tensor(Y_wi)
print(Y_w)

NameError: name 'Y' is not defined

In [ ]:
#Generates places
Y_p = []
for i in Y:
    place = [x*x for x in i]
    Y_p.append(place)
    
Y_p[1]

In [12]:
#validating model is new
for (i,j) in net.named_parameters():
    print(i,j)

fc1.weight Parameter containing:
tensor([[-0.0624,  0.0565,  0.0420,  ...,  0.0539, -0.0989,  0.1004],
        [ 0.0565, -0.0643, -0.0598,  ...,  0.1065,  0.0822, -0.1061],
        [ 0.0606,  0.0054,  0.0479,  ...,  0.0804, -0.0524, -0.0760],
        ...,
        [-0.0651,  0.0587, -0.0789,  ..., -0.0147, -0.0577,  0.0421],
        [-0.0010,  0.0184,  0.0802,  ...,  0.1016, -0.0702, -0.0289],
        [-0.0806, -0.0493, -0.0915,  ...,  0.0936, -0.0362, -0.0428]],
       device='cuda:0', requires_grad=True)
fc1.bias Parameter containing:
tensor([ 0.0506,  0.0928,  0.0255, -0.0845, -0.0155,  0.0240, -0.0922,  0.0384,
         0.1083,  0.0598, -0.0981,  0.0881, -0.0263,  0.0265, -0.0587, -0.0274,
         0.0884,  0.0539, -0.0761,  0.0500,  0.0291, -0.0653, -0.0530,  0.0864,
        -0.0548, -0.0148,  0.0668,  0.0223, -0.0342, -0.0175, -0.0149, -0.0148,
         0.0346, -0.0965,  0.0205, -0.0393, -0.0474, -0.0067, -0.0367,  0.0920,
         0.0508,  0.0969,  0.0130,  0.0338, -0.0377,  0.00

In [14]:
net = Net().to(device)

In [76]:
#utalized Win only class, not usefull for ranking placement

def train(net):
    BATCH_SIZE = 248
    EPOCHS = 10000
    loss_function = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=0.0001)

    for epoch in range(EPOCHS):
        for i in range(0, len(train_X), BATCH_SIZE): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            #print(f"{i}:{i+BATCH_SIZE}")
            batch_X = train_X[i:i+BATCH_SIZE]
            batch_y = train_y[i:i+BATCH_SIZE]

            #print(batch_X, batch_y)

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            net.zero_grad()
            optimizer.zero_grad()
            m = nn.LogSoftmax(dim = 1)
            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y.float())
            loss.backward()
            optimizer.step()    # Does the update



        #print(f"Epoch: {epoch}. Loss: {loss}")
        #test(net)
        if (epoch%100 == 0):
            print(f"Epoch: {epoch}. Loss: {loss}", end = " ")
            pred_list,_, _ = test(net)
            print(pd.Series(pred_list).value_counts())

In [79]:
train(net)

Epoch: 0. Loss: 0.0837092399597168 Accuracy:  0.2345    1464
6    1032
4     861
3     674
2     619
1     454
7     325
0     139
dtype: int64


In [74]:
def test(net):
    correct = 0
    total = 0
    realclasslist = []
    predictedClassList = []
    new_p_list = []
    with torch.no_grad():
        for i in range(len(test_X)):
            #print("test_y[i] \n", test_y[i], "\noutput y =", net(test_X[i]))
            real_class = torch.argmax(test_y[i]).to(device)
            realclasslist.append(test_y[i])
            net_out = net(test_X[i]).to(device)  # returns a list, 
            predictedClassList.append(net_out.tolist())
            #print(net_out)
            predicted_class = torch.argmax(net_out)
            #print("predicted_class = ", predicted_class,"\nreal class = ", real_class)
            new_p_list.append(predicted_class.item())
            #print(predicted_class)
            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total, 3), end="")
    return (new_p_list,predictedClassList, realclasslist)

In [57]:
#Test for indexed win only


def test(net):
    correct = 0
    total = 0
    realclasslist = []
    predictedClassList = []
    with torch.no_grad():
        for i in range(len(test_X)):
            #print("test_y[i] \n", test_y[i], "\noutput y =", net(test_X[i]))
            real_class = test_y[i]
            #print(real_class)
            realclasslist.append(test_y[i])
            net_out = net(test_X[i]).to(device)  # returns a list, 
            predictedClassList.append(net_out.tolist())
            #print(net_out)
            predicted_class = torch.argmax(net_out)
            #print("predicted_class = ", predicted_class,"\nreal class = ", real_class)
            
            #print(predicted_class)
            if predicted_class == real_class:
                correct += 1
            total += 1

    print("Accuracy: ", round(correct/total, 3))
    return (predictedClassList, realclasslist)

In [17]:
results = test(net)

Accuracy:  0.139


In [132]:
winners = [i.item() for i in Y_wi]
from collections import Counter
  
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]
    
List = winners
print(most_frequent(List))

4


In [ ]:
#TESTing
m = nn.LogSoftmax(dim = 1)

In [ ]:
for i in range(1000):
    print(torch.argmax(net(test_X)[i]))

In [99]:
df = pd.DataFrame({'Number' : Y_wi})